# Tokenize pre-mRNA sequences

In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset, Dataset
import multiprocessing as mp
import pandas as pd
import random
import json
import os

/opt/anaconda3/envs/torch201/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
os.chdir('/picb/rnasys2/zhouhanwen/github/LAMAR/')

In [3]:
# tokenizer
tokenizer_path = 'tokenizer/single_nucleotide/'
model_max_length = 1026
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, model_max_length=model_max_length)

In [4]:
def group_texts(examples):
    tokenized_inputs = tokenizer(examples["seq"], truncation=True, max_length=tokenizer.model_max_length)
    tokenized_inputs["labels"] = examples["label"]
    return tokenized_inputs

In [5]:
# paths of training set and testing set
train_set_path = 'SpliceSitePred/data/gencode_train.json'
# path of saving file
data_path = 'SpliceSitePred/data/ss_single_nucleotide'

In [6]:
with open(train_set_path) as f:
    train_set = json.load(f)

In [7]:
# This step takes ~30 mins. Please wait a moment. We will speed up this process.
# Not-known bug will report if use: dataset = load_dataset("json", data_files=train_set_path).
# We directly load the data in memory.
dataset = Dataset.from_list(train_set)
data = dataset.map(group_texts, batched=True, num_proc=mp.cpu_count())
data = data.remove_columns(["seq", "label"])
# data = data["train"].train_test_split(test_size=0.001, shuffle=True, seed=42)
data = data.train_test_split(test_size=0.001, shuffle=True, seed=42)
# Not-known bug will report if set num_proc=mp.cpu_count().
data.save_to_disk(data_path, num_proc=1)

Map (num_proc=160):   0%|          | 0/1022253 [00:00<?, ? examples/s]

Saving the dataset (0/28 shards):   0%|          | 0/1021230 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1023 [00:00<?, ? examples/s]

In [8]:
data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1021230
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1023
    })
})